In [ ]:
!pip install langchain
!pip install langchain-community
!pip install faiss-gpu

In [1]:
import json
total_data_list = []

with open(f'datasets/train_6.jsonl', 'r') as json_file:
    for line in json_file:
        data = json.loads(line)
        total_data_list.append(f"###질문: {data['input']} ###답변: {data['output']}")
len(total_data_list)

10000

In [3]:
import pandas as pd

pd.DataFrame(total_data_list).to_csv("datasets/train_data.csv",index=False,encoding='utf-8')

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

modelPath = "distiluse-base-multilingual-cased-v1"


model_kwargs = {'device':'cuda'}


encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='datasets/train_data.csv',encoding='utf-8')
data = loader.load()

In [11]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(data, embedding=embeddings)
db.save_local("faiss_index")

In [18]:
from langchain.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings

modelPath = "distiluse-base-multilingual-cased-v1"


model_kwargs = {'device':'cuda'}


encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = FAISS.load_local("faiss_index", embeddings)

retriever = db.as_retriever(search_kwargs={"k": 4})

In [13]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "beomi/OPEN-SOLAR-KO-10.7B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=400, no_repeat_ngram_size=3)
hf = HuggingFacePipeline(pipeline=pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 8/8 [00:11<00:00,  1.44s/it]


In [19]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """{context}\n당신은 전문 건축업자입니다. 다음 질문에 대해 답변해 주세요.\n### 질문: {question}\n\n### 답변: """
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | hf
    | StrOutputParser()
)

In [20]:
# 결과를 저장할 리스트 초기화
results_list = []

# RAG 체인 스트림을 통해 결과 생성 및 리스트에 저장
for chunk in rag_chain.stream("도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?"):
    print(chunk, end="", flush=True)
    results_list.append(chunk)

print(results_list)

KeyboardInterrupt: 